<a href="https://colab.research.google.com/github/JayjeetAtGithub/ANN-with-Keras/blob/master/road_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install requests

In [0]:
train_links = 'https://raw.githubusercontent.com/JayjeetAtGithub/Road-Segmentation/master/Data/_Links/Images.txt'
target_links = 'https://raw.githubusercontent.com/JayjeetAtGithub/Road-Segmentation/master/Data/_Links/Targets.txt'

In [0]:
import requests

# download train links into an array.
train_links_arr = requests.get(train_links).content.decode().split("\n")
print(train_links_arr)
# download target links into an array.
target_links_arr = requests.get(target_links).content.decode().split("\n")
print(target_links_arr)

['http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10078660_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10078675_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10078690_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10078705_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10078720_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10078735_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10078750_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10228660_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10228675_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10228705_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10228720_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/train/sat/10228735_15.tiff', 'http://www.cs.toronto.edu/~vmnih/data/mass_roads/t

In [0]:
import os
import shutil
import urllib.request

# prepare the download directory
# shutil.rmtree('./data')
os.makedirs('./data/train')
os.makedirs('./data/target')


In [0]:
# download train images
counter = 500
train_images_path = "./data/train/"
for link in train_links_arr[500:]:
  urllib.request.urlretrieve(link, os.path.join(train_images_path, f"train_{str(counter)}.tiff"))
  counter += 1

ValueError: ignored

In [0]:
# download train labels
counter = 500
target_images_path = "./data/target/"
for link in target_links_arr[500:]:
  urllib.request.urlretrieve(link, os.path.join(target_images_path, f"target_{str(counter)}.tiff"))
  counter += 1

ValueError: ignored

In [0]:
import numpy as np
import pickle
import cv2
from tqdm import tqdm_notebook
import os
import json
import math
import h5py


from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt

In [0]:
all_train_images = list()
all_target_images = list()

# load the training and target images into an array
for i in range(500, 1000):
  _img = cv2.imread(f"./data/train/train_{i}.tiff")
  _resized_img = cv2.resize(_img, (256, 256))
  all_train_images.append(_resized_img)

for i in range(500, 1000):
  img_ = cv2.imread(f"./data/target/target_{i}.tiff", 0)
  resized_img_ = cv2.resize(img_, (256, 256))
  all_target_images.append(resized_img_)

In [0]:
# convert into a numpy array
all_train_images = np.asarray(all_train_images)
all_target_images = np.asarray(all_target_images)
# check the shape of the dataset
print(all_train_images.shape)
print(all_target_images.shape)

# plt.imshow(all_train_images[8])
# plt.imshow(all_target_images[8])

(500, 256, 256, 3)
(500, 256, 256)


In [0]:
# create dir for storing compressed data
if not os.path.exists('./data/compressed'):
  os.makedirs('./data/compressed/train')
  os.makedirs('./data/compressed/target')

# create compressed training data
with h5py.File('./data/compressed/train/train_images.h5py', 'w') as hf:
        hf.create_dataset("all_train_images",  data=all_train_images)
    
# create compressed test data
with h5py.File('./data/compressed/target/target_images.h5py', 'w') as hf:
        hf.create_dataset("all_target_images",  data=all_target_images)

In [0]:
import warnings
warnings.filterwarnings('ignore')

import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import tensorflow as tf
import pandas as pd
from keras.models import Model, load_model
from skimage.morphology import label
import pickle
import tensorflow.keras.backend as K

from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import random
from skimage.io import imread, imshow, imread_collection, concatenate_images
from matplotlib import pyplot as plt
import h5py

seed = 56

Using TensorFlow backend.


In [0]:
# check the image and the masks file
image_file = './data/compressed/train/train_images.h5py'
mask_file = './data/compressed/target/target_images.h5py'
print(os.path.exists(image_file))
print(os.path.exists(mask_file))

True
True


In [0]:
# get images and masks from h5py files
hfile1 = h5py.File(image_file, 'r')
n1 = hfile1.get('all_train_images')
images = np.array(n1)
print(images.shape)
hfile1.close()

hfile2 = h5py.File(mask_file, 'r')
n2 = hfile2.get('all_target_images')
masks = np.array(n2)
print(masks.shape)
hfile2.close()

masks = np.expand_dims(masks, -1)

(500, 256, 256, 3)
(500, 256, 256)


In [0]:
# splitting data
from sklearn.model_selection import train_test_split
train_images, test_images, train_masks, test_masks = train_test_split(images, masks, test_size=0.2, random_state=seed)
del images, masks
print("TRAIN SET")
print(train_images.shape)
print(train_masks.shape)
print("TEST SET")
print(test_images.shape)
print(test_masks.shape)

TRAIN SET
(400, 256, 256, 3)
(400, 256, 256, 1)
TEST SET
(100, 256, 256, 3)
(100, 256, 256, 1)


In [0]:
from keras.models import Model, load_model
import tensorflow as tf
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import optimizers
from keras.layers import BatchNormalization
from tensorflow.keras.metrics import MeanIoU
import keras

In [0]:
inputs = Input((256, 256, 3))
s = Lambda(lambda x: x / 255) (inputs)

conv1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
conv1 = BatchNormalization() (conv1)
conv1 = Dropout(0.1) (conv1)
conv1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv1)
conv1 = BatchNormalization() (conv1)
pooling1 = MaxPooling2D((2, 2)) (conv1)

conv2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling1)
conv2 = BatchNormalization() (conv2)
conv2 = Dropout(0.1) (conv2)
conv2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv2)
conv2 = BatchNormalization() (conv2)
pooling2 = MaxPooling2D((2, 2)) (conv2)

conv3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling2)
conv3 = BatchNormalization() (conv3)
conv3 = Dropout(0.2) (conv3)
conv3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv3)
conv3 = BatchNormalization() (conv3)
pooling3 = MaxPooling2D((2, 2)) (conv3)

conv4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling3)
conv4 = BatchNormalization() (conv4)
conv4 = Dropout(0.2) (conv4)
conv4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv4)
conv4 = BatchNormalization() (conv4)
pooling4 = MaxPooling2D(pool_size=(2, 2)) (conv4)

conv5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (pooling4)
conv5 = BatchNormalization() (conv5)
conv5 = Dropout(0.3) (conv5)
conv5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv5)
conv5 = BatchNormalization() (conv5)


upsample6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (conv5)
upsample6 = concatenate([upsample6, conv4])
conv6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample6)
conv6 = BatchNormalization() (conv6)
conv6 = Dropout(0.2) (conv6)
conv6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv6)
conv6 = BatchNormalization() (conv6)

upsample7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (conv6)
upsample7 = concatenate([upsample7, conv3])
conv7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample7)
conv7 = BatchNormalization() (conv7)
conv7 = Dropout(0.2) (conv7)
conv7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv7)
conv7 = BatchNormalization() (conv7)

upsample8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (conv7)
upsample8 = concatenate([upsample8, conv2])
conv8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample8)
conv8 = BatchNormalization() (conv8)
conv8 = Dropout(0.1) (conv8)
conv8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv8)
conv8 = BatchNormalization() (conv8)

upsample9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (conv8)
upsample9 = concatenate([upsample9, conv1], axis=3)
conv9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (upsample9)
conv9 = BatchNormalization() (conv9)
conv9 = Dropout(0.1) (conv9)
conv9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv9)
conv9 = BatchNormalization() (conv9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv9)

model = Model(inputs=[inputs], outputs=[outputs])
model.summary()












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 256, 256, 16) 0         

In [0]:
EPOCHS = 100
LEARNING_RATE = 0.0001
BATCH_SIZE = 16


In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from datetime import datetime
tbc=TensorBoardColab()


Wait for 8 seconds...
TensorBoard link:
https://153c718e.ngrok.io


In [0]:
model_path = "./data/road_mapper.h5"

checkpointer = ModelCheckpoint(model_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystopper = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4)

In [0]:
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  
  return iou

def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [0]:
opt = keras.optimizers.adam(LEARNING_RATE)
model.compile(
      optimizer=opt,
      loss=soft_dice_loss,
      metrics=[iou_coef])

In [0]:
history = model.fit(train_images,
                    train_masks/255,
                    validation_split = 0.1,
                    epochs=EPOCHS,
                    batch_size = BATCH_SIZE,
                    callbacks = [checkpointer, earlystopper, lr_reducer, TensorBoardColabCallback(tbc)]
                       )




Train on 360 samples, validate on 40 samples


Epoch 1/100
360/360 [==============================] - 346s 961ms/step - loss: 0.8825 - iou_coef: 0.0606 - val_loss: 0.8501 - val_iou_coef: 0.0754

Epoch 00001: val_loss improved from inf to 0.85014, saving model to ./data/road_mapper.h5

Epoch 2/100
360/360 [==============================] - 333s 926ms/step - loss: 0.8577 - iou_coef: 0.0726 - val_loss: 0.8291 - val_iou_coef: 0.0850

Epoch 00002: val_loss improved from 0.85014 to 0.82912, saving model to ./data/road_mapper.h5
Epoch 3/100
360/360 [==============================] - 334s 928ms/step - loss: 0.8419 - iou_coef: 0.0806 - val_loss: 0.8121 - val_iou_coef: 0.0925

Epoch 00003: val_loss improved from 0.82912 to 0.81209, saving model to ./data/road_mapper.h5
Epoch 4/100
360/360 [==============================] - 334s 927ms/step - loss: 0.8324 - iou_coef: 0.0844 - val_loss: 0.7995 - val_iou_coef: 0.0986

Epoch 00004: val_loss improved from 0.81209 to 0.79948, saving model to ./data/r

In [0]:
model.save("./data/road_mapper_final.h5")


In [0]:
model = load_model("./data/road_mapper_final.h5", custom_objects={'soft_dice_loss': soft_dice_loss, 'iou_coef': iou_coef})

In [0]:
model.evaluate(test_images, test_masks)

In [0]:
predictions = model.predict(test_images, verbose=1)


In [0]:
thresh_val = 0.1
predicton_threshold = (predictions > thresh_val).astype(np.uint8)


In [0]:
ix = random.randint(0, len(predictions))
num_samples = 10

f = plt.figure(figsize = (15, 25))
for i in range(1, num_samples*4, 4):
  ix = random.randint(0, len(predictions))

  f.add_subplot(num_samples, 4, i)
  imshow(test_images[ix][:,:,0])
  plt.title("Image")
  plt.axis('off')

  f.add_subplot(num_samples, 4, i+1)
  imshow(np.squeeze(test_masks[ix][:,:,0]))
  plt.title("Groud Truth")
  plt.axis('off')

  f.add_subplot(num_samples, 4, i+2)
  imshow(np.squeeze(predictions[ix][:,:,0]))
  plt.title("Prediction")
  plt.axis('off')

  f.add_subplot(num_samples, 4, i+3)
  imshow(np.squeeze(predicton_threshold[ix][:,:,0]))
  plt.title("thresholded at {}".format(thresh_val))
  plt.axis('off')

plt.show()
